In [ ]:
!wget  https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz

--2023-10-18 09:21:21--  https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/4jw31k5mlzcmgis/genres.tar.gz [following]
--2023-10-18 09:21:22--  https://www.dropbox.com/s/raw/4jw31k5mlzcmgis/genres.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6eb63878c2c05c065e40d8351a.dl.dropboxusercontent.com/cd/0/inline/CF0F7YSqA5TfcZKhXIElQ3JH3durAGL-4d2SzWjNMWU7Hi2omrRpbzdEA4rU-9aV6lS_iWxw-67TkL-lwrV54jEjAgcSB3kQgkmQohsjDELu_drix_PluZUrwi6gpi-U8_s/file# [following]
--2023-10-18 09:21:22--  https://uc6eb63878c2c05c065e40d8351a.dl.dropboxusercontent.com/cd/0/inline/CF0F7YSqA5TfcZKhXIElQ3JH3durAGL-4d2SzWjNMWU7Hi2omrRpbzdEA4rU-9aV6lS_iWxw-67TkL-lwrV54jEjAgcSB3kQgkmQohsjDELu_drix_PluZUr

In [ ]:
#Unzipping the file ending with tar.gz

import tarfile

fname = 'genres.tar.gz'

if fname.endswith("tar.gz"):
    tar = tarfile.open(fname, "r:gz")
    tar.extractall()
    tar.close()
elif fname.endswith("tar"):
    tar = tarfile.open(fname, "r:")
    tar.extractall()
    tar.close()

In [ ]:
!pip install pydub
import pydub

In [ ]:
#Converting the file format from .au to .wav

from pydub import AudioSegment
import os


main_folder = '/content/genres'


subfolders = [subfolder for subfolder in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, subfolder))]


def convert_au_to_wav_and_replace(input_path):
    audio = AudioSegment.from_file(input_path, format="au")
    audio.export(input_path.replace('.au', '.wav'), format="wav")
    os.remove(input_path)


for subfolder in subfolders:
    subfolder_path = os.path.join(main_folder, subfolder)

    au_files = [file for file in os.listdir(subfolder_path) if file.endswith(".au")]


    for au_file in au_files:
        au_file_path = os.path.join(subfolder_path, au_file)
        convert_au_to_wav_and_replace(au_file_path)


In [ ]:
import json
import os
import math
import librosa

DATASET_PATH = "/content/genres"
JSON_PATH = "/content/data.json"
SAMPLE_RATE = 22050
TRACK_DURATION = 30
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION


def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):

    data = {
        "mappings": [],
        "labels": [],
        "mfcc": []
    }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)


    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        if dirpath is not dataset_path:    #Iterates through all genres and executes if the file paths for sub-genre and root-genre are different

            semantic_label = dirpath.split("/")[-1]
            data["mappings"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))

            for f in filenames:
                file_path = os.path.join(dirpath, f)
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)    #Loads all the data through librosa for each individual audio file


                for d in range(num_segments):   #Creates the mfcc features for each defined segment for each audio file.
                    begin = samples_per_segment * d
                    end = begin + samples_per_segment

                    mfcc = librosa.feature.mfcc(y=signal[begin:end], sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                    mfcc = mfcc.T

                    if len(mfcc) == num_mfcc_vectors_per_segment:   #If the size of mfcc's created matches the required mfcc's then the data is appended to json
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print("{}, segment:{}".format(file_path, d+1))

    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=5)


if __name__ == "__main__":
    save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)  #10 segments for each file hence the big list of outputs produced

Streaming output truncated to the last 5000 lines.
/content/genres/classical/classical.00003.wav, segment:7
/content/genres/classical/classical.00003.wav, segment:8
/content/genres/classical/classical.00003.wav, segment:9
/content/genres/classical/classical.00003.wav, segment:10
/content/genres/classical/classical.00082.wav, segment:1
/content/genres/classical/classical.00082.wav, segment:2
/content/genres/classical/classical.00082.wav, segment:3
/content/genres/classical/classical.00082.wav, segment:4
/content/genres/classical/classical.00082.wav, segment:5
/content/genres/classical/classical.00082.wav, segment:6
/content/genres/classical/classical.00082.wav, segment:7
/content/genres/classical/classical.00082.wav, segment:8
/content/genres/classical/classical.00082.wav, segment:9
/content/genres/classical/classical.00082.wav, segment:10
/content/genres/classical/classical.00069.wav, segment:1
/content/genres/classical/classical.00069.wav, segment:2
/content/genres/classical/classical

In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

DATA_PATH = "/content/data.json"


def load_data(data_path):

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])     #MFCC's are the input variables to the model, labels are the target
    y = np.array(data["labels"])

    return X, y



def prepare_datasets(test_size, validation_size):


    X, y = load_data(DATA_PATH)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)


    X_train = X_train[..., np.newaxis]            #Here we add new axis to match the input dimension for 2D conv network
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test


In [ ]:
def build_model(input_shape):

    model = keras.Sequential()

    model.add(keras.layers.Conv2D(218, (4, 4), activation='relu', padding='same', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((4, 4), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    model.add(keras.layers.Conv2D(125, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu', padding='same', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.Dense(10, activation='softmax'))

    return model


In [ ]:
if __name__ == "__main__":


    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)


    input_shape = (X_train.shape[1], X_train.shape[2], 1)
    model = build_model(input_shape)


    optimiser = keras.optimizers.Adam(learning_rate=0.0009)
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()


    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print('\nTest accuracy:', test_acc)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 130, 13, 218)      3706      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 65, 7, 218)        0         
 g2D)                                                            
                                                                 
 batch_normalization_4 (Bat  (None, 65, 7, 218)        872       
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           (None, 65, 7, 125)        245375    
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 33, 4, 125)        0         
 g2D)                                                            
                                                      

In [ ]:

def predict(model, X, y):

    X = X[ np.newaxis,...]

    prediction = model.predict(X)

    predicted_index = np.argmax(prediction, axis=1)

    print("Target: {}, Predicted label: {}".format(y, predicted_index))


X_to_predict = X_test[45]   #Since we are predicting for a single sample hence X_test[0] would be 1
y_to_predict = y_test[45]

predict(model, X_to_predict, y_to_predict)    #As you can see, the audio file at index 45 has been correctly classified as having the genre label 8.

1/1 [==============================] - 0s 32ms/step
Target: 8, Predicted label: [8]
